# Imports

In [2]:
# As usual, a bit of setup
import time
from time import process_time
import numpy as np
import matplotlib.pyplot as plt
import LOUPE.WILLOW.loupe as lp
import tensorflow as tf
import h5py
import pandas as pd
import csv
import copy
import math
from utils.data_utils import *
import sys
import re
from utils.spj import Config
from utils.spj import SPJ
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Specify Model Directory

In [12]:
home_dir = "/home/martnzjulio_a/songze"
#home_dir = "/home/songzeli"
version = "keep_drop_1"
minibatch_size = 1

print()
print("DIRECTORY SET TO: ", home_dir)
print("VERSION SET TO  : ", version)


DIRECTORY SET TO:  /home/martnzjulio_a/songze
VERSION SET TO  :  keep_drop_1


# Load Vocabulary

In [4]:
# Load Vocabulary
embedding_size =512
pad_len, num_steps = 50, 50
vocabulary,vocab_size = caption_preprocess(home_dir)
emb_matrix,word2id,id2word = get_wordvector(embedding_size,vocab_size,vocabulary)
num_classes = len(word2id)

# Word Embedding Matrix
emb_matrix, word2id, id2word = get_wordvector(embedding_size,vocab_size,vocabulary) #changed by Songze

Total number of words in all captions:  532264
Vocabulary Size (Unique):  11125


# Load All Data

In [7]:
# Load Training Data
train_file = home_dir + "/SPJ/train_2400.csv"
train_ids,train_data,train_padded_proposals,train_padded_framestamps = video_preprocess(home_dir, train_file)

# Train Captions
train_padded_sentences,train_padded_sentences_2,train_padded_sentences_id = get_padded_sentences_id(pad_len, train_ids, train_data, word2id) 
Ycaptions_train = np.transpose(copy.deepcopy(train_padded_sentences_2),axes=(0,2,1)).astype(np.int32)
Ycaptions_train = Ycaptions_train[:,:,1:]
Xcaptions_train = np.transpose(copy.deepcopy(train_padded_sentences),axes=(0,2,1)).astype(np.int32)


# Train Features 
VideoIds_train = train_ids
Framestamps_train = train_padded_framestamps
H_train = train_padded_proposals.astype(np.float32)
Ipast_train = temporal_indicator(train_padded_framestamps, mode="past").astype(np.float32)
Ifuture_train = temporal_indicator(train_padded_framestamps, mode="future").astype(np.float32)

num_train = len(train_ids)
print("Number of Training Examples:", num_train)
print()
print("VideoIds_train.shape: ", VideoIds_train.shape)
print("Framestamps_train.shape: ", Framestamps_train.shape)
print("Xcaptions_train.shape: ", Xcaptions_train.shape)
print("Ycaptions_train.shape: ", Ycaptions_train.shape)
print("H_train.shape: ", H_train.shape)
print("Ipast_train.shape: ", Ipast_train.shape)
print("Ifuture_train.shape: ", Ifuture_train.shape)
print()

Number of Training Examples: 2392

VideoIds_train.shape:  (2392,)
Framestamps_train.shape:  (2392, 2, 30)
Xcaptions_train.shape:  (2392, 30, 50)
Ycaptions_train.shape:  (2392, 30, 50)
H_train.shape:  (2392, 500, 30)
Ipast_train.shape:  (2392, 30, 30)
Ifuture_train.shape:  (2392, 30, 30)



In [8]:
# Load Validation Data
val_file = home_dir + "/SPJ/train_val_300.csv"
val_ids,val_data,val_padded_proposals,val_padded_framestamps = video_preprocess(home_dir, val_file)

# Train Captions
val_padded_sentences,val_padded_sentences_2,val_padded_sentences_id = get_padded_sentences_id(pad_len, val_ids, val_data, word2id) 
Ycaptions_val = np.transpose(copy.deepcopy(val_padded_sentences_2),axes=(0,2,1)).astype(np.int32)
Ycaptions_val = Ycaptions_val[:,:,1:]
Xcaptions_val = np.transpose(copy.deepcopy(val_padded_sentences),axes=(0,2,1)).astype(np.int32)


# Train Features 
VideoIds_val = val_ids
Framestamps_val = val_padded_framestamps
H_val = val_padded_proposals.astype(np.float32)
Ipast_val = temporal_indicator(val_padded_framestamps, mode="past").astype(np.float32)
Ifuture_val = temporal_indicator(val_padded_framestamps, mode="future").astype(np.float32)

num_val = len(val_ids)
print("Number of Validation Examples:", num_val)
print()
print("VideoIds_val.shape: ", VideoIds_val.shape)
print("Framestamps_val.shape: ", Framestamps_val.shape)
print("Xcaptions_val.shape: ", Xcaptions_val.shape)
print("Ycaptions_val.shape: ", Ycaptions_val.shape)
print("H_val.shape: ", H_val.shape)
print("Ipast_val.shape: ", Ipast_val.shape)
print("Ifuture_val.shape: ", Ifuture_val.shape)
print()

Number of Validation Examples: 300

VideoIds_val.shape:  (300,)
Framestamps_val.shape:  (300, 2, 30)
Xcaptions_val.shape:  (300, 30, 50)
Ycaptions_val.shape:  (300, 30, 50)
H_val.shape:  (300, 500, 30)
Ipast_val.shape:  (300, 30, 30)
Ifuture_val.shape:  (300, 30, 30)



# Training Function

In [13]:
def model(all_train, all_val, starter_learning_rate, keep_prob, num_epochs, home_dir, version, print_cost = True):
    """
    Implements a tensorflow neural network: C3D->ATTENTION->CAPTIONING
    
    Arguments:
    H_train -- training set, of shape = [n_train,num_c3d_features,num_proposals]
    Y_train -- caption labels, of shape = [n_train,num_proposals,num_steps+1]
    H_test -- training set, of shape = [n_test,num_c3d_features,num_proposals]
    Y_test -- caption labels, of shape = [n_test,num_proposals,num_steps+1]
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    (VideoIds_train, Framestamps_train, H_train, Ipast_train, Ifuture_train, Ycaptions_train, Xcaptions_train) = all_train
    (VideoIds_val, Framestamps_val, H_val,   Ipast_val,   Ifuture_val,   Ycaptions_val,   Xcaptions_val)   = all_val
    
    # Directory to Save Checkpoint
    checkpoint_dir = home_dir + "/checkpoints_" + str(version) + "/"
    tensorboard_dir =  home_dir + "/tensorboard_" + str(version) + "/"
    print("Checkpoint directory: ", checkpoint_dir)
    print("Tensorboard directory: ", tensorboard_dir)
    
    # Reset Graph
    tf.reset_default_graph()    
    
    # For Consistency
    tf.set_random_seed(1)                             
    seed = 3                                         
    
    # Number of Training Examples
    num_train = H_train.shape[0] 
    num_val = H_val.shape[0] 
    
    # to keep track of costs
    costs = []
    
    
    # Model
    config = Config()
    spj = SPJ(config)
    
    # Print Hyperparameters
    print()
    print("Hyperparameters:")
    print("----------------")
    print("Starter Learning Rate: ", starter_learning_rate)
    print("Number of Proposals: ", spj.config.num_proposals)
    print("C3D Features Dim: ", spj.config.num_c3d_features )
    print("Batch Size: ", spj.config.batch_size)
    print("Dropout Keep Prob: ", keep_prob)
    print("Vocab Size: ", spj.config.num_classes)
    print("Number of LSTM Time Steps: ", spj.config.num_steps)
    print("Word Embedding Size: " , spj.config.hidden_dim)
    print("LSTM Hidden Dim: " , spj.config.hidden_dim)
    print("LSTM Num Layers: " , spj.config.num_layers)
    
    # Global Epoch Number
    global_step = tf.Variable(0, name='global_step', trainable=False)

    # Learning Rate Decay
    learning_rate = tf.train.exponential_decay(
        learning_rate = starter_learning_rate, 
        global_step = global_step,
        decay_steps = 100000, 
        decay_rate = 0.96, 
        staircase=True)
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(spj.loss, global_step=global_step)
    
    # Passing global_step to minimize() will increment it at each step.
    learning_step = (optimizer)
    
    # Initialize all the variables
    init = tf.global_variables_initializer()
    
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver(max_to_keep=5)
    
    # Tensorboard Loss
    training_summary = tf.summary.scalar("training_loss", spj.loss)
    validation_summary = tf.summary.scalar("validation_loss", spj.loss)
    #writer = tf.train.SummaryWriter(...)
    

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
    
        # check for latest checkpoint
        latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
        if latest_checkpoint == None:
            # If no check point run the initialization
            print()
            print("No checkpoint exists, initializing parameters...")
            sess.run(init)
        else:
            print()
            print("Restoring from latest checkpoint...")
            saver.restore(sess, latest_checkpoint)
        
        #Tensorboard
        summary_writer = tf.summary.FileWriter(tensorboard_dir,sess.graph)
        
        # Training Loop
        print()
        start = process_time() # Record start time
        for epoch in range(num_epochs):
            
            # Variable to store cost
            epoch_train_loss = 0.0
            epoch_val_loss = 0.0
            train_losses = []
            val_losses = []
            
            # Get minibatches
            num_train_minibatches = num_train // spj.config.batch_size 
            num_val_minibatches = num_val // spj.config.batch_size 
            seed = seed + 1
            train_minibatches = random_mini_batches(VideoIds_train, Framestamps_train, H_train, Ipast_train, Ifuture_train, Ycaptions_train, Xcaptions_train, spj.config.batch_size , seed)
            val_minibatches = random_mini_batches(VideoIds_val, Framestamps_val, H_val, Ipast_val,   Ifuture_val,   Ycaptions_val,   Xcaptions_val,   spj.config.batch_size , seed)

            for counter, train_minibatch in enumerate(train_minibatches):
                
                # Select minibatch
                (minibatch_VideoIds_train, minibatch_Framestamps_train, minibatch_H_train, minibatch_Ipast_train, minibatch_Ifuture_train, minibatch_Ycaptions_train, minibatch_Xcaptions_train) = train_minibatch
                minibatch_Ycaptions_train = id_2_one_hot_void_padding(minibatch_Ycaptions_train, spj.config.num_classes, void_dim=0)
                
                # Run Train Session
                train_feed={spj._H: minibatch_H_train, 
                      spj._Ipast: minibatch_Ipast_train, 
                      spj._Ifuture: minibatch_Ifuture_train, 
                      spj._x: minibatch_Xcaptions_train, 
                      spj._y: minibatch_Ycaptions_train, 
                      spj._keep_prob: keep_prob}
                _ , minibatch_train_loss, train_summ = sess.run([optimizer, spj.loss, training_summary], feed_dict=train_feed)
                train_losses.append(minibatch_train_loss)
                
            for counter, val_minibatch in enumerate(val_minibatches):
                
                # Select minibatch
                (minibatch_VideoIds_val, minibatch_Framestamps_val, minibatch_H_val, minibatch_Ipast_val, minibatch_Ifuture_val, minibatch_Ycaptions_val, minibatch_Xcaptions_val) = val_minibatch
                minibatch_Ycaptions_val = id_2_one_hot_void_padding(minibatch_Ycaptions_val, spj.config.num_classes, void_dim=0)
                
                # Run Validation Session
                val_feed={spj._H: minibatch_H_val, 
                          spj._Ipast: minibatch_Ipast_val, 
                          spj._Ifuture: minibatch_Ifuture_val, 
                          spj._x: minibatch_Xcaptions_val, 
                          spj._y: minibatch_Ycaptions_val, 
                          spj._keep_prob: 1.0}
                minibatch_val_loss, val_summ = sess.run([spj.loss, validation_summary], feed_dict=val_feed) #
                val_losses.append(minibatch_val_loss)
            
            epoch_train_loss = np.mean(train_losses)
            epoch_val_loss = np.mean(val_losses)
            
            # Print cost
            if print_cost == True:
                global_epoch = tf.train.global_step(sess, global_step)//num_train_minibatches
                print("Epoch: ", global_epoch)
                print ("Training Loss: ", epoch_train_loss)
                print ("Validation Loss: ", epoch_val_loss)
                # Add and Write to Tensorboard
                train_summary = tf.Summary()
                val_summary = tf.Summary()
                train_summary.value.add(tag="train_losss", simple_value=epoch_train_loss)
                train_summary.value.add(tag="val_losss", simple_value=epoch_val_loss)
                summary_writer.add_summary(train_summary, global_epoch)
                summary_writer.add_summary(val_summary, global_epoch)

            
            # Save Model (every 20 epochs)
            if global_epoch % 10 == 0:
                print("Saving Checkpoint for global_step " + str(global_epoch))
                saver.save(sess, checkpoint_dir + 'model', global_step = global_epoch)
        
        # Save and Print Processed Time
        end = process_time() 
        print()
        print("Time Elapased: ", end - start)
        
        return 0

# Run Training

In [14]:
# Train Model
learning_rate = 0.001
keep_prob = 1.0
num_epochs = 1000
all_train = (VideoIds_train, Framestamps_train, H_train, Ipast_train, Ifuture_train, Ycaptions_train, Xcaptions_train)
all_val =   (VideoIds_val, Framestamps_val, H_val,   Ipast_val,   Ifuture_val,   Ycaptions_val,   Xcaptions_val)
execute = model(all_train, all_val, learning_rate, keep_prob, num_epochs, home_dir, version)

Checkpoint directory:  /home/martnzjulio_a/songze/checkpoints_keep_drop_1/
Tensorboard directory:  /home/martnzjulio_a/songze/tensorboard_keep_drop_1/

Hyperparameters:
----------------
Starter Learning Rate:  0.001
Number of Proposals:  30
C3D Features Dim:  500
Batch Size:  1
Dropout Keep Prob:  1.0
Vocab Size:  10194
Number of LSTM Time Steps:  50
Word Embedding Size:  512
LSTM Hidden Dim:  512
LSTM Num Layers:  2

No checkpoint exists, initializing parameters...



IndexError: index 10215 is out of bounds for axis 1 with size 10194